In [ ]:
%%time
import pandas as pd
from sklearn import cross_validation
from sklearn import ensemble
from sklearn.decomposition import RandomizedPCA
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import accuracy_score
#from sklearn.feature_selection import SelectKBest
#from sklearn.naive_bayes import GaussianNB
#from sklearn import tree
#from sklearn.svm import SVC
#from sklearn.naive_bayes import BernoulliNB
#from datetime import datetime

In [1]:
%%time
import numpy as np

def apk(actual, predicted, k=5):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    Computes the average precision at k.
    This function computes the average precision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if score <= 0:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=5):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

CPU times: user 70.2 ms, sys: 88.5 ms, total: 159 ms
Wall time: 562 ms


In [ ]:
%%time
train_dtypes = {'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'is_booking': pd.np.int64,
'cnt': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64,
'hotel_cluster': pd.np.int64}

In [ ]:
%%time
all_train = pd.read_csv('train.csv', dtype=train_dtypes, iterator=True, chunksize=1000)
#iter_csv = pandas.read_csv('file.csv', iterator=True, chunksize=1000)
all_train = pd.concat([chunk[chunk['is_booking'] == 1] for chunk in all_train])
#temp_train = pd.read_csv('train.csv', dtype=train_dtypes, iterator=True, chunksize=1000)
#all_train = pd.concat(temp_train, ignore_index=True)

In [ ]:
%%time
all_train['orig_destination_distance'] = all_train['orig_destination_distance'].fillna(-1)
all_train['date_time'] = pd.to_datetime(all_train['date_time'], errors='coerce')
all_train['srch_ci'] = pd.to_datetime(all_train['srch_ci'], errors='coerce')
all_train['srch_co'] = pd.to_datetime(all_train['srch_co'], errors='coerce')
all_train['activity_month'] = all_train['date_time'].fillna(-1).dt.month.astype(int)
all_train['checkin_month'] = all_train['srch_ci'].fillna(-1).dt.month.astype(int)
all_train['checkout_month'] = all_train['srch_co'].fillna(-1).dt.month.astype(int)
#Split groups into two different classifiers for destinations vs. no destinations

In [ ]:
%%time
destinations = pd.read_csv('destinations.csv')

In [ ]:
%%time
#features = all_train[['activity_month', 'user_location_country', 'user_location_region', 'user_location_city', 'hotel_market', 'orig_destination_distance', 'checkin_month', 'checkout_month']]
features = all_train.drop(['is_booking', 'cnt', 'user_id', 'hotel_cluster', 'date_time', 'srch_ci', 'srch_co'],1)
labels = all_train['hotel_cluster']
features = features.reindex_axis(sorted(features.columns), axis=1)
#features = features.values

In [ ]:
%%time
with_dest_match = pd.merge(features, destinations)
wo_dest_match = pd.merge(features, destinations, how='left')
wo_dest_match = wo_dest_match[feat_dest.isnull().any(axis=1)]
wo_dest_match = wo_dest_match.dropna(axis=1)

In [ ]:
all_scores = []

In [ ]:
%%time
for i in range(0, 20):
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(features, labels, test_size=0.5)
    clf = tree.DecisionTreeClassifier(min_samples_split=500)
    clf = clf.fit(features_train, labels_train.values.ravel())
    pred_probs = pd.DataFrame(clf.predict_proba(features_test))
    pred_probs = pd.DataFrame([list([r.sort_values(ascending=False)[:3].index.values]) for i,r in pred_probs.iterrows()])
    labels_test_df = pd.DataFrame(labels_test.values, index=range(0, len(labels_test)))
    test_score = mapk(labels_test_df.values, pred_probs.values)
    all_scores.append(test_score)

In [ ]:
print np.mean(all_scores)
#all_scores = np.asarray(all_scores)
#all_scores.mean()

In [ ]:
# %%time
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(features, labels, test_size=0.5)
# selector = SelectKBest(k=10)
# selector.fit(features_train, labels_train.values.ravel())
# features_train = selector.transform(features_train)

In [ ]:
%%time
#clf = LogisticRegression(tol=0.1)
#clf = tree.DecisionTreeClassifier(min_samples_split=500)
#clf = GaussianNB()
#clf = ensemble.AdaBoostClassifier()
#clf = ensemble.AdaBoostClassifier(SVC(probability=True, kernel='linear'),n_estimators=10)
#clf = ensemble.GradientBoostingClassifier(SVC(probability=True, kernel='linear'),n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
#clf = ensemble.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0, verbose=3)
clf = ensemble.RandomForestClassifier(n_estimators=20, min_samples_split=100, max_features=10)
#clf - BernoulliNB()
clf = clf.fit(features_train, labels_train.values.ravel())

In [ ]:
# %%time
# features_test = selector.transform(features_test)

In [ ]:
%%time
pred_probs = pd.DataFrame(clf.predict_proba(features_test))

In [ ]:
%%time
pred_probs = pd.DataFrame([list([r.sort_values(ascending=False)[:5].index.values]) for i,r in pred_probs.iterrows()])

In [ ]:
%%time
labels_test_df = pd.DataFrame(labels_test.values, index=range(0, len(labels_test)))

In [ ]:
%%time
test_score = mapk(labels_test_df.values, pred_probs.values)
print test_score

In [ ]:
#print accuracy_score(pred, labels_test.values.ravel())

In [ ]:
%%time
test_dtypes = {'id': pd.np.int64,
'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64}

In [ ]:
%%time
all_test = pd.read_csv('test.csv', dtype=test_dtypes)

In [ ]:
%%time
all_test['orig_destination_distance'] = all_test['orig_destination_distance'].fillna(-1)
all_test['date_time'] = pd.to_datetime(all_test['date_time'], errors='coerce')
all_test['srch_ci'] = pd.to_datetime(all_test['srch_ci'], errors='coerce')
all_test['srch_co'] = pd.to_datetime(all_test['srch_co'], errors='coerce')
all_test['activity_month'] = all_test['date_time'].fillna(-1).dt.month.astype(int)
all_test['checkin_month'] = all_test['srch_ci'].fillna(-1).dt.month.astype(int)
all_test['checkout_month'] = all_test['srch_co'].fillna(-1).dt.month.astype(int)

In [ ]:
%%time
#testing_features = all_test[['activity_month', 'user_location_country', 'user_location_region', 'user_location_city', 'hotel_market', 'orig_destination_distance', 'checkin_month', 'checkout_month']]
testing_features = all_test.drop(['user_id', 'id', 'date_time', 'srch_ci', 'srch_co'],1)
testing_features = testing_features.reindex_axis(sorted(testing_features.columns), axis=1)
#testing_features = testing_features.values

In [ ]:
tst_feat_dest = pd.merge(testing_features, destinations)

In [ ]:
print len(tst_feat_dest), len(testing_features)

In [ ]:
# %%time
# testing_list = selector.transform(testing_features)

In [ ]:
%%time
#feature_test_file = selector.transform(all_test)
#pred = clf.predict(feature_test_list)
test_probs = pd.DataFrame(clf.predict_proba(testing_features))

In [ ]:
%%time
test_probs = pd.Series([(i, r.sort_values(ascending=False)[:5].index.values) for i,r in test_probs.iterrows()])
test_probs = test_probs.values

In [ ]:
%%time
indices = [a for a,b in test_probs]
values = [b for a,b in test_probs]

In [ ]:
%%time
submission = pd.DataFrame()
submission['id'] = indices
submission['hotel_cluster'] = [' '.join(str(x) for x in y) for y in values]

In [ ]:
%%time
submission.to_csv('submission.csv', index=False)